In [105]:
import psycopg2
import gensim
from collections import namedtuple, Counter

model = gensim.models.doc2vec.Doc2Vec.load('../models/doc2vec_2k_primary_key')

with psycopg2.connect(dbname='arxiv') as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM articles")        
        col_names = [col.name for col in cur.description]
        Article = namedtuple("Article", col_names)
        hits = 0
        for row in cur:
            article = Article(*row)
            relateds = model.docvecs.most_similar(positive=[article.index], topn=2)
            related_ids = tuple([idx for idx, similarity in relateds])
            with conn.cursor() as newcur:
                newcur.execute("SELECT subject FROM articles WHERE index in %s"%(related_ids, ))
                cats = newcur.fetchall()
                new_hits = [1 if cat==article.subject else 0 for cat in cats]
                hits += sum(new_hits)

print(hits)

SELECT * FROM articles WHERE index=200


[(200,
  'Riemann curvature of a boosted spacetime geometry',
  'Battista, Emmanuele|Esposito, Giampiero|Scudellaro, Paolo|Tramontano, Francesco',
  'High Energy Physics - Theory',
  "The ultrarelativistic boosting procedure had been applied in the literature to map the metric of Schwarzschild-de Sitter spacetime into a metric describing de Sitter spacetime plus a shock-wave singularity located on a null hypersurface. This paper evaluates the Riemann curvature tensor of the boosted Schwarzschild-de Sitter metric by means of numerical calculations, which make it possible to reach the ultrarelativistic regime gradually by letting the boost velocity approach the speed of light. Thus, for the first time in the literature, the singular limit of curvature through Dirac's delta distribution and its derivatives is numerically evaluated for this class of spacetimes. Eventually, the analysis of the Kretschmann invariant and the geodesic equation show that the spacetime possesses a scalar curvatu

(Column(name='index', type_code=23, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='title', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='authors', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='subject', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='abstract', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None),
 Column(name='last_submitted', type_code=1082, display_size=None, internal_size=4, precision=None, scale=None, null_ok=None),
 Column(name='arxiv_id', type_code=25, display_size=None, internal_size=-1, precision=None, scale=None, null_ok=None))